# BLEU

https://github.com/moses-smt/mosesdecoder/blob/master/scripts/generic/multi-bleu.perl

## Results

From: <a href="../paper/webnlg2017/The WebNLG Challenge -  Generating Text from RDF Data.pdf">The WebNLG Challenge - Generating Text from RDF Data</a>

<img src='../img/bleu_result.jpg' />

## Melbourne

<a href="../data/webnlg2017/submissions/melbourne/WebNLG Challenge - Bayu Distiawan T.pdf">report</a>

In [1]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/GKB_Unimelb_all-cat.txt

BLEU = 45.13, 79.9/55.4/38.8/27.4 (BP=0.969, ratio=0.969, hyp_len=40642, ref_len=41921)


In [3]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/GKB_Unimelb_all-cat.txt | tail -10

castle is english language .
eric flint was born in burbank , california .
macmillan publishers is the parent company of farrar , straus and giroux .
john cowper powys was a glastonbury romance .
soho press is in united states .
the secret scripture was published by faber and faber .
asian americans are an ethnic group in united states .
english language is spoken in united states .
weymouth sands was preceded by a glastonbury romance .
the manager of a . c . chievo verona is rolando maran .


## Tilb-SMT

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [16]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-2_all-cat.txt | tail -10

novel castle is english language
. eric flint was born in burbank , california .
macmillan publishers is the parent company of farrar , straus and giroux
john cowper powys notablework a glastonbury romance
soho press , united states .
, published by faber and faber
asian americans are an ethnic group .
english is the language of the united states .
weymouth sands was preceded by a glastonbury romance
the manager of a . c . chievo verona is rolando maran


In [15]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-2_all-cat.txt

BLEU = 44.28, 74.4/53.0/37.5/26.8 (BP=0.992, ratio=0.992, hyp_len=42193, ref_len=42521)


## PKUWriter

<a href="../data/webnlg2017/submissions/pkuwriter/PKUWriter_system.pdf">report</a>

In [2]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/PKUWriter_all-cat.txt | tail -10

the novel into battle is written by the english language .
john cowper powys was born in burbank , california .
the parent company of the macmillan publishers press is the macmillan publishers company of the macmillan publishers .
the novel owen glendower is a notable work by john cowper powys .
soho press is located in united states .
the polish academy for writtenwork is referencenumber _ in _ the _ national _ register _ of _ historic _ places .
asian americans are an ethnic group in the united states .
english is the language spoken in the united states .
the novel into battle is preceded by a glastonbury romance .
rolando maran manages the a . c . chievo verona .


In [5]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/PKUWriter_all-cat.txt

BLEU = 39.88, 73.9/49.9/35.6/26.0 (BP=0.927, ratio=0.930, hyp_len=38248, ref_len=41143)


## UPF-FORGe

<a href="../data/webnlg2017/submissions/upf/WebNLG_V0.2.pdf">report</a>

In [4]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UPF-TALN_all-cat.txt | tail -10

the english language is spoken in castle ( novel ) .
eric flint was born in burbank ( california ) .
the parent company of farrar , straus and giroux is macmillan publishers .
a glastonbury romance wrote john cowper powys .
soho press is published in the united states .
the publisher of the secret scripture is faber and faber .
a ethnic group of the united states are asian americans .
the english language is spoken in the united states .
weymouth sands follows a glastonbury romance .
rolando maran is the manager of a . c . chievo verona .


In [6]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UPF-TALN_all-cat.txt

BLEU = 38.65, 75.5/48.7/31.1/19.5 (BP=1.000, ratio=1.064, hyp_len=47234, ref_len=44405)


## Tilb-Pipeline

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [5]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-3_all-cat.txt | tail -10

castle is in english .
eric flint was born in burbank , california .
macmillan publishers owns farrar , straus , and giroux .
a glastonbury romance is a notable work by john cowper powys .
soho press is in united states .
the secret scripture publisher of faber and faber .
asian americans live in united states .
united states is in english .
weymouth sands was preceded by a glastonbury romance .
rolando maran manages a . c . chievo verona .


In [8]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-3_all-cat.txt

BLEU = 35.29, 81.8/56.6/38.8/26.5 (BP=0.755, ratio=0.781, hyp_len=30335, ref_len=38854)


## Tilb-NMT

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [6]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-1_all-cat.txt | tail -10

english is the language of castle .
eric flint was born in burbank , california .
macmillan publishers is the parent company of farrar , straus , and giroux .
a glastonbury romance is a notable work by john cowper powys .
soho press is located in united states .
th secret scriptures was published by faber and faber .
asian americans are an ethnic group in united states .
english is the language of united states .
weymouth sands was preceded by a glastonbury romance .
the manager of a . c . chievo verona is rolando maran .


In [9]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-1_all-cat.txt

BLEU = 34.60, 68.8/42.7/27.5/17.7 (BP=1.000, ratio=1.054, hyp_len=45605, ref_len=43278)


## Baseline

In [7]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Baseline_all-cat.txt | tail -10

the english language is spoken in the republic of ireland .
author was born in burbank , california .
the parent company of wiley - blackwell is macmillan publishers .
a glastonbury romance was written by author .
lahore is in the united states .
the parent company of wiley - blackwell is parentcompany .
asian americans are an ethnic group in the united states .
english is the language of the united states .
a glastonbury romance was preceded by precededby .
the manager of a . c . chievo verona is rolando maran .


In [10]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Baseline_all-cat.txt

BLEU = 33.24, 65.4/42.3/31.8/24.8 (BP=0.865, ratio=0.873, hyp_len=35312, ref_len=40435)


## Adapt

<a href="../data/webnlg2017/submissions/adaptCenter/WebNLG_system.pdf">report</a>

In [8]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/ADAPT_Centre_all-cat.txt | tail -10

the novel castle is the language spoken in the english language .
eric flint was born in burbank , california .
macmillan publishers is the parent company of farrar , straus and giroux .
a glastonbury romance is the notable work of john cowper powys .
soho press is located in the united states .
the novel the secret scriptures , is published by faber and faber .
asian americans are an ethnic group in the u . s .
english is the language of the united states .
a glastonbury romance is the sequel to weymouth sands .
the manager of a . c . chievo verona is rolando maran .


In [11]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/ADAPT_Centre_all-cat.txt

BLEU = 31.06, 54.9/35.7/25.3/18.7 (BP=1.000, ratio=1.337, hyp_len=59727, ref_len=44663)


## UIT-VNU

<a href="../data/webnlg2017/submissions/uit-danglt-clnlp/Phong-Nguyen-Tran_Dang-Tuan-Nguyen.docx">report</a>

In [9]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UIT-DANGNT-CLNLP_all-cat.txt | tail -10

english language is one of the languages that is spoken in castle ( novel ) .
the birth place of eric flint is burbank , california .
macmillan publishers is the parent company of the farrar , straus and giroux .
null
soho press is located within the country of the united states .
the book the secret scripture was published by faber and faber .
asian americans are part of one of the ethnic groups in the united states .
english language is one of the languages that is spoken in united states .
a glastonbury romance was written prior to weymouth sands .
rolando maran is the manager of the a . c . chievo verona .


In [12]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UIT-DANGNT-CLNLP_all-cat.txt

BLEU = 7.07, 67.5/54.4/39.5/29.2 (BP=0.156, ratio=0.350, hyp_len=11219, ref_len=32072)


# METEOR

http://www.cs.cmu.edu/~alavie/METEOR/

Download meteor-1.5 and unpack into ../evaluation/webnlg2017/

The jar path must be ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar

## Results

From: <a href="../paper/webnlg2017/The WebNLG Challenge -  Generating Text from RDF Data.pdf">The WebNLG Challenge - Generating Text from RDF Data</a>

<img src='../img/meteor_result.jpg' />

## UPF-FORGe

<a href="../data/webnlg2017/submissions/upf/WebNLG_V0.2.pdf">report</a>

In [5]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UPF-TALN_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz | tail -10

Test words:             47234
Reference words:        42200
Chunks:                 14422
Precision:              0.7391270509045015
Recall:                 0.8031416242379965
f1:                     0.7698058188555684
fMean:                  0.7928416187351017
Fragmentation penalty:  0.5067403511305503

Final score:            0.3910767784663624


## Tilb-SMT

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [6]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-2_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz | tail -10

Test words:             42193
Reference words:        42317
Chunks:                 11454
Precision:              0.6986051446343317
Recall:                 0.7644854923281655
f1:                     0.7300620815229181
fMean:                  0.7538223701991189
Fragmentation penalty:  0.49528804709412466

Final score:            0.38046316060733304


## Melbourne

<a href="../data/webnlg2017/submissions/melbourne/WebNLG Challenge - Bayu Distiawan T.pdf">report</a>

In [7]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/GKB_Unimelb_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz | tail -10

Test words:             40666
Reference words:        42030
Chunks:                 12139
Precision:              0.7721945285853984
Recall:                 0.7449236133412855
f1:                     0.7583139671715875
fMean:                  0.7488908010393752
Fragmentation penalty:  0.497395080474511

Final score:            0.3763962007897742


## Tilb-NMT

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [8]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-1_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz | tail -10

Test words:             45605
Reference words:        42143
Chunks:                 13486
Precision:              0.6629704974017897
Recall:                 0.7226106746263194
f1:                     0.6915070268797431
fMean:                  0.7129896963141423
Fragmentation penalty:  0.510814499480972

Final score:            0.34878422145634347


## Adapt

<a href="../data/webnlg2017/submissions/adaptCenter/WebNLG_system.pdf">report</a>

In [9]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/ADAPT_Centre_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz | tail -10

Test words:             59727
Reference words:        43177
Chunks:                 13045
Precision:              0.5086640217307478
Recall:                 0.6676700499074749
f1:                     0.5774205490828529
fMean:                  0.6377656591777211
Fragmentation penalty:  0.5045522048796571

Final score:            0.315979589643074


## PKUWriter

<a href="../data/webnlg2017/submissions/pkuwriter/PKUWriter_system.pdf">report</a>

In [10]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/PKUWriter_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz | tail -10

Test words:             38248
Reference words:        42618
Chunks:                 11252
Precision:              0.7107364891009933
Recall:                 0.6261925845196739
f1:                     0.6657913688978063
fMean:                  0.6375686451682541
Fragmentation penalty:  0.5021382567514663

Final score:            0.3174210371240728


## Tilb-Pipeline

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [11]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-3_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz | tail -10

Test words:             30335
Reference words:        41846
Chunks:                 9683
Precision:              0.7952671826196668
Recall:                 0.5943403590944575
f1:                     0.6802775153496506
fMean:                  0.6177519115958529
Fragmentation penalty:  0.500867362329304

Final score:            0.3083401410609527


## Baseline

In [12]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Baseline_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz | tail -10

Test words:             35326
Reference words:        42886
Chunks:                 9941
Precision:              0.5854253097193214
Recall:                 0.4659432085368059
f1:                     0.5188950257360138
fMean:                  0.48065816355637425
Fragmentation penalty:  0.5101800648231838

Final score:            0.23543595051539076


## UIT-VNU

<a href="../data/webnlg2017/submissions/uit-danglt-clnlp/Phong-Nguyen-Tran_Dang-Tuan-Nguyen.docx">report</a>

In [13]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UIT-DANGNT-CLNLP_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz | tail -10

Test words:             11219
Reference words:        43444
Chunks:                 2534
Precision:              0.638817898022893
Recall:                 0.1714209246827397
f1:                     0.2703073506581485
fMean:                  0.1925534492216448
Fragmentation penalty:  0.4847845962347004

Final score:            0.09920650308713085


# TER

http://www.cs.umd.edu/~snover/tercom/

Download tercom-0.7.25 and unpack into ../evaluation/webnlg2017/

The jar path must be ../evaluation/webnlg2017/tercom-0.7.25/tercom-0.7.25.jar

## Results

From: <a href="../paper/webnlg2017/The WebNLG Challenge -  Generating Text from RDF Data.pdf">The WebNLG Challenge - Generating Text from RDF Data</a>

<img src='../img/ter_result.jpg' />

## Melbourne

<a href="../data/webnlg2017/submissions/melbourne/WebNLG Challenge - Bayu Distiawan T.pdf">report</a>

In [11]:
!java -jar ../evaluation/webnlg2017/tercom-0.7.25/tercom.7.25.jar -r ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.ter -h ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/GKB_Unimelb_all-cat_ter.txt | tail -5

Processing id1862:1
Total TER: 0.47440303246041 (20546.0/43309.16666666672)
Number of calls to beam search: 233776
Number of segments scored: 4894
Number of shifts tried: 228882


## Tilb-SMT

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [12]:
!java -jar ../evaluation/webnlg2017/tercom-0.7.25/tercom.7.25.jar -r ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.ter -h ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-2_all-cat_ter.txt | tail -5

Processing id1862:1
Total TER: 0.537692174481922 (23287.0/43309.16666666672)
Number of calls to beam search: 266866
Number of segments scored: 4894
Number of shifts tried: 261972


## PKUWriter

<a href="../data/webnlg2017/submissions/pkuwriter/PKUWriter_system.pdf">report</a>

In [13]:
!java -jar ../evaluation/webnlg2017/tercom-0.7.25/tercom.7.25.jar -r ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.ter -h ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/PKUWriter_all-cat_ter.txt | tail -5

Processing id1862:1
Total TER: 0.5530007119355018 (23950.0/43309.16666666672)
Number of calls to beam search: 181913
Number of segments scored: 4894
Number of shifts tried: 177019


## UPF-FORGe

<a href="../data/webnlg2017/submissions/upf/WebNLG_V0.2.pdf">report</a>

In [14]:
!java -jar ../evaluation/webnlg2017/tercom-0.7.25/tercom.7.25.jar -r ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.ter -h ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UPF-TALN_all-cat_ter.txt | tail -5

Processing id1862:1
Total TER: 0.5593273171576449 (24224.0/43309.16666666672)
Number of calls to beam search: 349335
Number of segments scored: 4894
Number of shifts tried: 344441


## Tilb-Pipeline

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [15]:
!java -jar ../evaluation/webnlg2017/tercom-0.7.25/tercom.7.25.jar -r ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.ter -h ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-3_all-cat_ter.txt | tail -5

Processing id1862:1
Total TER: 0.566023359181081 (24514.0/43309.16666666672)
Number of calls to beam search: 181333
Number of segments scored: 4175
Number of shifts tried: 177158


## Tilb-NMT

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [16]:
!java -jar ../evaluation/webnlg2017/tercom-0.7.25/tercom.7.25.jar -r ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.ter -h ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-1_all-cat_ter.txt | tail -5

Processing id1862:1
Total TER: 0.6095014527332543 (26397.0/43309.16666666672)
Number of calls to beam search: 249034
Number of segments scored: 4894
Number of shifts tried: 244140


## Baseline

In [17]:
!java -jar ../evaluation/webnlg2017/tercom-0.7.25/tercom.7.25.jar -r ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.ter -h ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Baseline_all-cat_ter.txt | tail -5

Processing id1862:1
Total TER: 0.6130803717457806 (26552.0/43309.16666666672)
Number of calls to beam search: 134473
Number of segments scored: 4894
Number of shifts tried: 129579


## UIT-VNU

<a href="../data/webnlg2017/submissions/uit-danglt-clnlp/Phong-Nguyen-Tran_Dang-Tuan-Nguyen.docx">report</a>

In [18]:
!java -jar ../evaluation/webnlg2017/tercom-0.7.25/tercom.7.25.jar -r ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.ter -h ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UIT-DANGNT-CLNLP_all-cat_ter.txt | tail -5

Processing id1862:1
Total TER: 0.8274922552962218 (35838.0/43309.16666666672)
Number of calls to beam search: 33824
Number of segments scored: 4894
Number of shifts tried: 28930


## Adapt

<a href="../data/webnlg2017/submissions/adaptCenter/WebNLG_system.pdf">report</a>

In [19]:
!java -jar ../evaluation/webnlg2017/tercom-0.7.25/tercom.7.25.jar -r ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.ter -h ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/ADAPT_Centre_all-cat_ter.txt | tail -5

Processing id1862:1
Total TER: 0.8492659367724297 (36781.0/43309.16666666672)
Number of calls to beam search: 213881
Number of segments scored: 4894
Number of shifts tried: 208987
